In [1]:
import json
import time
import pandas
import requests
from bs4 import BeautifulSoup

In [2]:
!pip install trafilatura
import os
import re
import shutil
import trafilatura

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 3.8 MB/s eta 0:00:00


In [3]:
url = "https://www.stonerealestate.com.au"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")
soup

<!DOCTYPE html>

<html dir="ltr" itemscope="" itemtype="https://schema.org/WebPage" lang="en-US" xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml">
<head profile="https://gmpg.org/xfn/11">
<meta content="width=device-width, minimum-scale=1.0, maximum-scale=1.0, user-scalable=yes" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<script type="text/javascript">
/* <![CDATA[ */
 var gform;gform||(document.addEventListener("gform_main_scripts_loaded",function(){gform.scriptsLoaded=!0}),window.addEventListener("DOMContentLoaded",function(){gform.domLoaded=!0}),gform={domLoaded:!1,scriptsLoaded:!1,initializeOnLoaded:function(o){gform.domLoaded&&gform.scriptsLoaded?o():!gform.domLoaded&&gform.scriptsLoaded?window.addEventListener("DOMContentLoaded",o):document.addEventListener("gform_main_scripts_loaded",o)},hooks:{action:{},filter:{}},addAction:function(o,n,r,t){gform.addHook("action",o,n,r,t)},addFilter:function(o,n,r,t)

In [4]:
from urllib.parse import urljoin, urlparse

def get_stone_realestate_urls(soup, base_url="https://www.stonerealestate.com.au"):
    urls = set()

    for a in soup.find_all("a", href=True):
        href = a["href"].strip()
        urls.add(href)
        # print(href)

        # # Convert relative URLs to absolute
        # full_url = urljoin(base_url, href)

        # parsed = urlparse(full_url)

        # # Check domain
        # if parsed.netloc == "https://www.stonerealestate.com.au":
        #     urls.add(full_url)

    return sorted(urls)

urls = get_stone_realestate_urls(soup)

for url in urls:
    print(url)

#
http://thestoneshop.com.au/
https://stone.lightspeedvt.com/
https://stone.supplycrew.com.au/
https://www.agentbox.com.au/features-pages/websites
https://www.facebook.com/pages/Stone-Real-Estate/123695347694910
https://www.instagram.com/STONE_Real_Estate/
https://www.linkedin.com/company/stone-real-estate-?trk=tyah&trkInfo=clickedVertical%3Acompany%2CclickedEntityId%3A6405155%2Cidx%3A3-2-7%2CtarId%3A1450153664292%2Ctas%3AStone+real+estate
https://www.lookatmystrata.com.au/login/966851
https://www.stonerealestate.com.au
https://www.stonerealestate.com.au/about/
https://www.stonerealestate.com.au/acreage/
https://www.stonerealestate.com.au/blog/
https://www.stonerealestate.com.au/buy/
https://www.stonerealestate.com.au/buy/acreage/
https://www.stonerealestate.com.au/buy/commercial/
https://www.stonerealestate.com.au/buy/off-market-sales/
https://www.stonerealestate.com.au/buy/open-sales/
https://www.stonerealestate.com.au/buy/property-alerts/
https://www.stonerealestate.com.au/buy/resid

In [5]:
url = "https://www.stonerealestate.com.au/wp-content/plugins/zoorealty/display/elements/load_listings.php?post_id=116&status=1,4&list=ResidentialSale&order=&order_direction=&property_type=&keywords=&price_min=&price_max=&bedrooms=&bathrooms=&carspaces=&sur_suburbs=&related_office=&office_id=&exclude_office_id=&page=17"
res = requests.get(url)
res.text

'\t\r\n\t\t\t<div class="column listing">\r\n\t\r\n\t\t\t\t<div class="container">\r\n\t\r\n\t\t\t\t\t<div class="image">\r\n\t\t\t\t\t\t<a class="centerimage" href="https://www.stonerealestate.com.au/property/8566886-16-1684-channel-highway-margate-tas/" title="Margate 16/1684 Channel Highway">\r\n\t\t\t\t\t\t\t<img class="fit-height" src="https://phimg.reapit.website/bfdc4d00f9a79f51d5c09c0fc195d2ea693676d8" alt="Margate 16/1684 Channel Highway" />\r\n\t\t\t\t\t\t</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t<div class="sticker underoffer"  style="display:block" >Under offer</div>\r\n\t\t\t\t\t\t\t\t\t\t\t</div>\r\n\t\r\n\t\t\t\t\t<div class="details">\r\n\t\t\r\n\t\t\t\t\t\t<div class="title">\r\n\t\t\t\t\t\t\t<p class="suburb"><a href="https://www.stonerealestate.com.au/property/8566886-16-1684-channel-highway-margate-tas/" title="Margate">Margate</a>&nbsp;</p>\r\n\t\t\t\t\t\t\t<p class="address">16/1684 Channel Highway&nbsp;</p>\r\n\t\t\t\t\t\t</div>\r\n\t\t\r\n\t\t\t\t\t\t<p class="rooms">\r

In [6]:
from bs4 import BeautifulSoup

def parse_property_listings(soup):
    listings = []

    for card in soup.select("div.column.listing"):
        data = {}

        # --- URL & Image ---
        link = card.select_one(".image a.centerimage")
        data["url"] = link["href"] if link else None

        img = card.select_one(".image img")
        data["image_url"] = img["src"] if img else None

        # --- Status (Under offer etc.) ---
        sticker = card.select_one(".sticker")
        data["status"] = sticker.get_text(strip=True) if sticker else None

        # --- Location ---
        data["suburb"] = (
            card.select_one(".title .suburb a").get_text(strip=True)
            if card.select_one(".title .suburb a")
            else None
        )

        data["address"] = (
            card.select_one(".title .address").get_text(strip=True)
            if card.select_one(".title .address")
            else None
        )

        # --- Rooms ---
        rooms = card.select(".rooms .item")
        data["bedrooms"] = None
        data["bathrooms"] = None
        data["carspaces"] = None
        data["has_study"] = False

        for r in rooms:
            text = r.get_text(strip=True)
            icon = r.select_one("i")

            if not icon:
                continue

            cls = " ".join(icon.get("class", []))

            if "icon-bedrooms" in cls:
                data["bedrooms"] = int(text.split()[0])
            elif "icon-bathrooms" in cls:
                data["bathrooms"] = int(text.split()[0])
            elif "icon-carspaces" in cls:
                data["carspaces"] = int(text.split()[0])
            elif "icon-study" in cls:
                data["has_study"] = True

        # --- Price & Open Times ---
        price_block = card.select_one(".price")

        data["price"] = None
        data["open_times"] = None

        if price_block:
            default_price = price_block.select_one(".default-price")
            if default_price:
                data["price"] = default_price.get_text(strip=True)

            open_amount = price_block.select_one(".opens-price .amount")
            if open_amount:
                open_text = open_amount.get_text(strip=True)
                if open_text:
                    data["open_times"] = open_text

        listings.append(data)

    return listings

url = "https://www.stonerealestate.com.au/wp-content/plugins/zoorealty/display/elements/load_listings.php?post_id=116&status=1,4&list=ResidentialSale&order=&order_direction=&property_type=&keywords=&price_min=&price_max=&bedrooms=&bathrooms=&carspaces=&sur_suburbs=&related_office=&office_id=&exclude_office_id=&page=17"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")
properties = parse_property_listings(soup)

for p in properties:
    print(p)

{'url': 'https://www.stonerealestate.com.au/property/8566886-16-1684-channel-highway-margate-tas/', 'image_url': 'https://phimg.reapit.website/bfdc4d00f9a79f51d5c09c0fc195d2ea693676d8', 'status': 'Under offer', 'suburb': 'Margate', 'address': '16/1684 Channel Highway', 'bedrooms': 2, 'bathrooms': 1, 'carspaces': 2, 'has_study': False, 'price': 'Offers over $525,000', 'open_times': None}
{'url': 'https://www.stonerealestate.com.au/property/8566857-10-1-3-wentworth-street-manly-nsw/', 'image_url': 'https://phimg.reapit.website/6c6a00f77695dc14f3891d4d005bad87ad8a2159', 'status': None, 'suburb': 'Manly', 'address': '10/1-3 Wentworth Street', 'bedrooms': 2, 'bathrooms': 2, 'carspaces': 2, 'has_study': False, 'price': 'Auction', 'open_times': None}
{'url': 'https://www.stonerealestate.com.au/property/8566856-4-8-james-street-manly-nsw/', 'image_url': 'https://phimg.reapit.website/6732b07574cfee6e6b9cf7d462bc1f9c3b7f5e19', 'status': None, 'suburb': 'Manly', 'address': '4/8 James Street', 'be

In [ ]:
import requests
from bs4 import BeautifulSoup

BASE_URL = (
    "https://www.stonerealestate.com.au/wp-content/plugins/zoorealty/"
    "display/elements/load_listings.php"
)

PARAMS = {
    "post_id": "111",
    "status": "1,4",
    "list": "lease",
    "order": "",
    "order_direction": "",
    "property_type": "",
    "keywords": "",
    "price_min": "",
    "price_max": "",
    "bedrooms": "",
    "bathrooms": "",
    "carspaces": "",
    "sur_suburbs": "",
    "related_office": "",
    "office_id": "",
    "exclude_office_id": "",
}

all_properties = []
page = 1

while True:
    PARAMS["page"] = page

    print(f"Fetching page {page}...")

    res = requests.get(BASE_URL, params=PARAMS, timeout=15)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, "html.parser")
    properties = parse_property_listings(soup)

    # Stop if no properties returned
    if not properties:
        print("No properties found, stopping.")
        break

    all_properties.extend(properties)

    print(f"→ Found {len(properties)} properties")

    # Stop condition: last page has fewer than 12 items
    if len(properties) < 12:
        print("Last page reached.")
        break

    page += 1

print(f"\nTotal properties collected: {len(all_properties)}")


Fetching page 1...
→ Found 12 properties
Fetching page 2...
→ Found 12 properties
Fetching page 3...
→ Found 12 properties
Fetching page 4...
→ Found 12 properties
Fetching page 5...
→ Found 12 properties
Fetching page 6...
→ Found 12 properties
Fetching page 7...
→ Found 12 properties
Fetching page 8...
→ Found 12 properties
Fetching page 9...
→ Found 12 properties
Fetching page 10...
→ Found 12 properties
Fetching page 11...
→ Found 12 properties
Fetching page 12...
→ Found 12 properties
Fetching page 13...
→ Found 12 properties
Fetching page 14...
→ Found 12 properties
Fetching page 15...
→ Found 12 properties
Fetching page 16...
→ Found 12 properties
Fetching page 17...
→ Found 12 properties
Fetching page 18...
→ Found 12 properties
Fetching page 19...
→ Found 12 properties
Fetching page 20...
→ Found 12 properties
Fetching page 21...
→ Found 12 properties
Fetching page 22...
→ Found 12 properties
Fetching page 23...
→ Found 12 properties
Fetching page 24...
→ Found 12 properties
F

In [ ]:
import json

output_filename = 'lease.json'
with open(output_filename, 'w') as f:
    json.dump(all_properties, f, indent=4)

print(f"Successfully saved {len(all_properties)} properties to {output_filename}")

Successfully saved 455 properties to lease.json


In [ ]:
pages = [
  "https://www.stonerealestate.com.au",
  "https://www.stonerealestate.com.au/about/",
  "https://www.stonerealestate.com.au/acreage/",
  "https://www.stonerealestate.com.au/blog/",
  "https://www.stonerealestate.com.au/buy/",
  "https://www.stonerealestate.com.au/buy/acreage/",
  "https://www.stonerealestate.com.au/buy/commercial/",
  "https://www.stonerealestate.com.au/buy/off-market-sales/",
  "https://www.stonerealestate.com.au/buy/open-sales/",
  "https://www.stonerealestate.com.au/buy/property-alerts/",
  "https://www.stonerealestate.com.au/buy/residential/",
  "https://www.stonerealestate.com.au/careers/",
  "https://www.stonerealestate.com.au/franchise/",
  "https://www.stonerealestate.com.au/manage/",
  "https://www.stonerealestate.com.au/our-approach-to-management/",
  "https://www.stonerealestate.com.au/our-approach-to-selling/",
  "https://www.stonerealestate.com.au/our-people/join-us/",
  "https://www.stonerealestate.com.au/our-people/our-agents/",
  "https://www.stonerealestate.com.au/our-people/our-offices/",
  "https://www.stonerealestate.com.au/privacy-policy/",
  "https://www.stonerealestate.com.au/projects/",
  "https://www.stonerealestate.com.au/property-report/",
  "https://www.stonerealestate.com.au/property/4394568-18-arrowgrass-street-aberglasslyn-nsw/",
  "https://www.stonerealestate.com.au/property/5028950-5-foxtail-street-fern-bay-nsw/",
  "https://www.stonerealestate.com.au/property/6412043-369a-bexley-road-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/6757984-56a-geoffrey-road-chittaway-point-nsw/",
  "https://www.stonerealestate.com.au/property/7155103-38-nicoll-street-roselands-nsw/",
  "https://www.stonerealestate.com.au/property/7336529-1-21-heyes-street-gillieston-heights-nsw/",
  "https://www.stonerealestate.com.au/property/7348819-17-10-16-hegerty-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/7430147-81a-melbourne-street-aberdare-nsw/",
  "https://www.stonerealestate.com.au/property/7556158-2-muirfield-way-medowie-nsw/",
  "https://www.stonerealestate.com.au/property/7694555-3-64-66-connemarra-street-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/7729937-18-3-gahnia-place-hamlyn-terrace-nsw/",
  "https://www.stonerealestate.com.au/property/7796595-1-14-aspect-drive-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/7807294-4-4-rocklands-road-wollstonecraft-nsw/",
  "https://www.stonerealestate.com.au/property/7845916-3316-2-rothschild-avenue-rosebery-nsw/",
  "https://www.stonerealestate.com.au/property/7968305-34-trevally-street-korora-nsw/",
  "https://www.stonerealestate.com.au/property/7982724-92-94-talavera-road-macquarie-park-nsw/",
  "https://www.stonerealestate.com.au/property/8009527-7-28-marine-parade-miami-qld/",
  "https://www.stonerealestate.com.au/property/8046624-75-amalfi-drive-isle-of-capri-qld/",
  "https://www.stonerealestate.com.au/property/8083353-44-1-9-beach-street-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8213526-1-4-10-the-boulevarde-brighton-le-sands-nsw/",
  "https://www.stonerealestate.com.au/property/8223674-9a-chelsea-street-balmoral-nsw/",
  "https://www.stonerealestate.com.au/property/8231572-31-chico-street-copacabana-nsw/",
  "https://www.stonerealestate.com.au/property/8248230-4b-earl-street-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8252194-56-the-corso-gorokan-nsw/",
  "https://www.stonerealestate.com.au/property/8265745-3-5-pitt-owen-avenue-arncliffe-nsw/",
  "https://www.stonerealestate.com.au/property/8266979-24-table-top-road-north-avoca-nsw/",
  "https://www.stonerealestate.com.au/property/8292238-196-chatham-street-hamilton-south-nsw/",
  "https://www.stonerealestate.com.au/property/8298945-51-harbourside-crescent-port-macquarie-nsw/",
  "https://www.stonerealestate.com.au/property/8304850-242-252-pitt-street-merrylands-nsw/",
  "https://www.stonerealestate.com.au/property/8305600-23a-frances-street-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8321749-155-riverview-street-riverview-nsw/",
  "https://www.stonerealestate.com.au/property/8359142-1-113-oceano-street-copacabana-nsw/",
  "https://www.stonerealestate.com.au/property/8376018-2-driver-place-wyong-nsw/",
  "https://www.stonerealestate.com.au/property/8383586-2-25-29-oconnell-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8384670-1-45-murdoch-street-cremorne-nsw/",
  "https://www.stonerealestate.com.au/property/8385953-26-peterborough-avenue-lake-illawarra-nsw/",
  "https://www.stonerealestate.com.au/property/8387628-309-38-44-pembroke-street-epping-nsw/",
  "https://www.stonerealestate.com.au/property/8393917-1-strangers-creek-place-bella-vista-nsw/",
  "https://www.stonerealestate.com.au/property/8393918-21-strangers-creek-place-bella-vista-nsw/",
  "https://www.stonerealestate.com.au/property/8393919-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8402928-204-8-12-good-street-westmead-nsw/",
  "https://www.stonerealestate.com.au/property/8402929-205-8-12-good-street-westmead-nsw/",
  "https://www.stonerealestate.com.au/property/8413793-9-kateena-avenue-tascott-nsw/",
  "https://www.stonerealestate.com.au/property/8420151-34-brett-place-west-pennant-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8420222-413-yarramalong-road-wyong-creek-nsw/",
  "https://www.stonerealestate.com.au/property/8420427-95-aloha-drive-chittaway-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8439122-1a-thomas-mitchell-road-killarney-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8445435-3604-11-hassall-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8445542-1-22-24-wagawn-street-tugun-qld/",
  "https://www.stonerealestate.com.au/property/8446755-36-lennox-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8447566-7b-alicia-road-mount-kuring-gai-nsw/",
  "https://www.stonerealestate.com.au/property/8449656-37b-griffiths-street-sans-souci-nsw/",
  "https://www.stonerealestate.com.au/property/8449690-210-25-mann-street-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8451176-64-buff-point-avenue-buff-point-nsw/",
  "https://www.stonerealestate.com.au/property/8452993-204-10-west-promenade-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8454332-8-mcmaster-place-little-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8456581-5-35-scenic-highway-terrigal-nsw/",
  "https://www.stonerealestate.com.au/property/8458858-34-montego-court-eatons-hill-qld/",
  "https://www.stonerealestate.com.au/property/8462727-23-ponsonby-parade-seaforth-nsw/",
  "https://www.stonerealestate.com.au/property/8465008-658-15-thompsons-road-pokolbin-nsw/",
  "https://www.stonerealestate.com.au/property/8465561-g21-4-seven-street-epping-nsw/",
  "https://www.stonerealestate.com.au/property/8465829-30-dampier-boulevard-killarney-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8465842-3-32-dening-street-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8469612-88-katherine-avenue-amaroo-act/",
  "https://www.stonerealestate.com.au/property/8470970-16-lampard-circuit-bruce-act/",
  "https://www.stonerealestate.com.au/property/8471682-37-dunmore-street-north-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/8472024-59-burdekin-avenue-amaroo-act/",
  "https://www.stonerealestate.com.au/property/8472332-206-208-oak-road-matcham-nsw/",
  "https://www.stonerealestate.com.au/property/8472472-16-colblack-close-tacoma-nsw/",
  "https://www.stonerealestate.com.au/property/8475837-94-highgate-street-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/8480334-87-mabo-boulevard-bonner-act/",
  "https://www.stonerealestate.com.au/property/8480520-9-12-linda-street-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8480894-70a-wattle-tree-road-holgate-nsw/",
  "https://www.stonerealestate.com.au/property/8481846-16-14-jenkins-street-collaroy-nsw/",
  "https://www.stonerealestate.com.au/property/8482037-8-jasmine-street-horsley-nsw/",
  "https://www.stonerealestate.com.au/property/8491007-23-fishermans-parade-daleys-point-nsw/",
  "https://www.stonerealestate.com.au/property/8492181-1-2-pape-street-franklin-act/",
  "https://www.stonerealestate.com.au/property/8492576-1001-16-18-seventh-avenue-palm-beach-qld/",
  "https://www.stonerealestate.com.au/property/8493350-55b-high-street-thirroul-nsw/",
  "https://www.stonerealestate.com.au/property/8493709-2-3-7-gladstone-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8493718-302-107-chandos-street-crows-nest-nsw/",
  "https://www.stonerealestate.com.au/property/8494031-60-spinnaker-ridge-way-belmont-nsw/",
  "https://www.stonerealestate.com.au/property/8494831-40-134-shoalhaven-street-kiama-nsw/",
  "https://www.stonerealestate.com.au/property/8494983-7-judith-street-seaforth-nsw/",
  "https://www.stonerealestate.com.au/property/8495130-2-bracken-close-berowra-nsw/",
  "https://www.stonerealestate.com.au/property/8496877-305-77-edward-street-carlton-nsw/",
  "https://www.stonerealestate.com.au/property/8496937-369-lakedge-avenue-berkeley-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8497047-30-ladera-drive-copacabana-nsw/",
  "https://www.stonerealestate.com.au/property/8497173-6-leopard-avenue-elanora-qld/",
  "https://www.stonerealestate.com.au/property/8499347-1-boomerang-street-lake-cathie-nsw/",
  "https://www.stonerealestate.com.au/property/8500373-6-68-smith-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8502183-40-zoriana-close-mardi-nsw/",
  "https://www.stonerealestate.com.au/property/8502655-26-quealy-street-casey-act/",
  "https://www.stonerealestate.com.au/property/8503773-46-2-brunswick-road-terrigal-nsw/",
  "https://www.stonerealestate.com.au/property/8503794-213-414-pacific-highway-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8504328-23-murray-road-beecroft-nsw/",
  "https://www.stonerealestate.com.au/property/8504335-7-sheoak-drive-buninyong-vic/",
  "https://www.stonerealestate.com.au/property/8505514-5-guyong-street-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8506386-22-18-20-market-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8506462-13-22-zelman-close-watanobbi-nsw/",
  "https://www.stonerealestate.com.au/property/8506661-16-curtis-parade-the-entrance-north-nsw/",
  "https://www.stonerealestate.com.au/property/8506747-283-lakedge-avenue-berkeley-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8509374-5-66-68-park-road-rydalmere-nsw/",
  "https://www.stonerealestate.com.au/property/8509651-5-amadeus-place-north-boambee-valley-nsw/",
  "https://www.stonerealestate.com.au/property/8509836-12-pinehurst-avenue-rouse-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8509888-1329-100-belmore-street-ryde-nsw/",
  "https://www.stonerealestate.com.au/property/8509980-21-brian-street-fairy-meadow-nsw/",
  "https://www.stonerealestate.com.au/property/8510162-3-10-armidale-street-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8510611-3-bolwarra-crescent-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8514302-7-amadeus-place-north-boambee-valley-nsw/",
  "https://www.stonerealestate.com.au/property/8515384-13-schuppan-street-oran-park-nsw/",
  "https://www.stonerealestate.com.au/property/8515394-level-3-311-2b-wharf-road-melrose-park-nsw/",
  "https://www.stonerealestate.com.au/property/8515593-130-lakedge-avenue-berkeley-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8516132-1-ralston-avenue-belrose-nsw/",
  "https://www.stonerealestate.com.au/property/8517253-15-karoo-avenue-east-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8517335-2-bromborough-road-roseville-nsw/",
  "https://www.stonerealestate.com.au/property/8517610-33-oratava-avenue-west-pennant-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8517612-204-7-gladstone-parade-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8517853-3-windsorgreen-drive-wyong-nsw/",
  "https://www.stonerealestate.com.au/property/8518133-27-spinks-road-east-corrimal-nsw/",
  "https://www.stonerealestate.com.au/property/8519075-12-83-85-alfred-street-ramsgate-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8520565-29-kirby-street-rydalmere-nsw/",
  "https://www.stonerealestate.com.au/property/8520865-102-seaforth-crescent-seaforth-nsw/",
  "https://www.stonerealestate.com.au/property/8521599-8a-arkana-street-telopea-nsw/",
  "https://www.stonerealestate.com.au/property/8522646-8-endeavour-street-sans-souci-nsw/",
  "https://www.stonerealestate.com.au/property/8522718-9-2-6-bowen-street-chatswood-nsw/",
  "https://www.stonerealestate.com.au/property/8523144-101-hills-street-north-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8523342-58-scenic-highway-terrigal-nsw/",
  "https://www.stonerealestate.com.au/property/8523353-114-40-cotlew-street-east-southport-qld/",
  "https://www.stonerealestate.com.au/property/8524184-42-41-chandler-street-belconnen-act/",
  "https://www.stonerealestate.com.au/property/8524471-12-94-henry-kendall-street-franklin-act/",
  "https://www.stonerealestate.com.au/property/8526151-27-bath-street-thirroul-nsw/",
  "https://www.stonerealestate.com.au/property/8526404-39-bronhill-street-moncrieff-act/",
  "https://www.stonerealestate.com.au/property/8526810-31-cambridge-avenue-windsor-nsw/",
  "https://www.stonerealestate.com.au/property/8527649-33-orana-avenue-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8528553-246-bathurst-street-sawyers-gully-nsw/",
  "https://www.stonerealestate.com.au/property/8528856-22-15-21-webb-avenue-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8528861-6-61-broken-bay-road-ettalong-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8529370-14-tamora-close-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8530350-202b-3-11-burleigh-street-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8530674-14-anvil-avenue-elderslie-nsw/",
  "https://www.stonerealestate.com.au/property/8531251-8-wolseley-road-mcgraths-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8531335-12a-gosford-avenue-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8531560-12-470-472-pacific-highway-asquith-nsw/",
  "https://www.stonerealestate.com.au/property/8531714-120-gooraway-drive-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8532044-5-2-6-russell-avenue-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8532241-192-parraweena-road-miranda-nsw/",
  "https://www.stonerealestate.com.au/property/8534268-41-19-23-herbert-street-st-leonards-nsw/",
  "https://www.stonerealestate.com.au/property/8536176-5-ray-ellis-crescent-forde-act/",
  "https://www.stonerealestate.com.au/property/8536177-28-hedland-circuit-flynn-act/",
  "https://www.stonerealestate.com.au/property/8536354-60-west-burleigh-road-burleigh-heads-qld/",
  "https://www.stonerealestate.com.au/property/8536505-352-west-botany-street-brighton-le-sands-nsw/",
  "https://www.stonerealestate.com.au/property/8537005-12-greta-road-kulnura-nsw/",
  "https://www.stonerealestate.com.au/property/8537338-53-north-avoca-parade-north-avoca-nsw/",
  "https://www.stonerealestate.com.au/property/8537993-10-blaxland-road-killara-nsw/",
  "https://www.stonerealestate.com.au/property/8538000-1-west-hill-street-mcgraths-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8539189-37-hull-road-beecroft-nsw/",
  "https://www.stonerealestate.com.au/property/8539190-22-49-55-beamish-road-northmead-nsw/",
  "https://www.stonerealestate.com.au/property/8539704-9-phelp-court-amaroo-act/",
  "https://www.stonerealestate.com.au/property/8539706-4-wardell-place-agnes-banks-nsw/",
  "https://www.stonerealestate.com.au/property/8539757-7-jule-knight-street-casey-act/",
  "https://www.stonerealestate.com.au/property/8539802-4-benaroon-circuit-amaroo-act/",
  "https://www.stonerealestate.com.au/property/8539850-47a-malvern-avenue-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8539986-20-klaus-moje-street-whitlam-act/",
  "https://www.stonerealestate.com.au/property/8540182-109-quarry-road-rosewood-nsw/",
  "https://www.stonerealestate.com.au/property/8540760-14-62-lowanna-street-braddon-act/",
  "https://www.stonerealestate.com.au/property/8541226-62-orchard-road-kangy-angy-nsw/",
  "https://www.stonerealestate.com.au/property/8541684-18-134-shoalhaven-street-kiama-nsw/",
  "https://www.stonerealestate.com.au/property/8541745-9-amadeus-place-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8543302-14-bligh-street-silverwater-nsw/",
  "https://www.stonerealestate.com.au/property/8543511-144-macquarie-grove-caves-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8543601-82-malton-road-beecroft-nsw/",
  "https://www.stonerealestate.com.au/property/8544090-360-lower-colo-road-lower-portland-nsw/",
  "https://www.stonerealestate.com.au/property/8544294-7-reid-street-croki-nsw/",
  "https://www.stonerealestate.com.au/property/8544295-682-ocean-drive-north-haven-nsw/",
  "https://www.stonerealestate.com.au/property/8544365-326-blaxlands-ridge-road-blaxlands-ridge-nsw/",
  "https://www.stonerealestate.com.au/property/8544445-11-bogan-road-booker-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8544773-30-applecross-avenue-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8544873-176-geoffrey-road-chittaway-point-nsw/",
  "https://www.stonerealestate.com.au/property/8545204-11-dandarbong-avenue-carlingford-nsw/",
  "https://www.stonerealestate.com.au/property/8545424-2-522-524-pacific-highway-mount-colah-nsw/",
  "https://www.stonerealestate.com.au/property/8545547-33-st-johns-road-bradbury-nsw/",
  "https://www.stonerealestate.com.au/property/8545618-48-amiens-street-gladesville-nsw/",
  "https://www.stonerealestate.com.au/property/8545720-9-martins-creek-road-paterson-nsw/",
  "https://www.stonerealestate.com.au/property/8546189-7-43-47-mallett-street-camperdown-nsw/",
  "https://www.stonerealestate.com.au/property/8546291-64-71-83-smith-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8546395-43-primula-street-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8546887-39-bayview-street-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/8546891-4-hooton-street-forde-act/",
  "https://www.stonerealestate.com.au/property/8547175-10-52-bay-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8547346-27-little-lake-crescent-warilla-nsw/",
  "https://www.stonerealestate.com.au/property/8547390-8-tippet-place-quakers-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8547394-12-philip-charley-drive-port-macquarie-nsw/",
  "https://www.stonerealestate.com.au/property/8547539-31-connaghan-avenue-east-corrimal-nsw/",
  "https://www.stonerealestate.com.au/property/8547597-425-doveton-street-soldiers-hill-vic/",
  "https://www.stonerealestate.com.au/property/8548349-88b-howes-road-east-kurrajong-nsw/",
  "https://www.stonerealestate.com.au/property/8548702-317-43-arthur-blakeley-way-coombs-act/",
  "https://www.stonerealestate.com.au/property/8551197-198-victoria-avenue-chatswood-nsw/",
  "https://www.stonerealestate.com.au/property/8551210-88-tallebudgera-drive-palm-beach-qld/",
  "https://www.stonerealestate.com.au/property/8551644-34-1-nevertire-street-lawson-act/",
  "https://www.stonerealestate.com.au/property/8552028-241-williams-road-kulnura-nsw/",
  "https://www.stonerealestate.com.au/property/8552804-35-albert-drive-killara-nsw/",
  "https://www.stonerealestate.com.au/property/8552870-204-10-mentmore-avenue-rosebery-nsw/",
  "https://www.stonerealestate.com.au/property/8553188-110-8-baudinette-circuit-bruce-act/",
  "https://www.stonerealestate.com.au/property/8553263-4-53-east-esplanade-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8553937-1-sickles-drive-grasmere-nsw/",
  "https://www.stonerealestate.com.au/property/8554041-404-wendouree-parade-lake-wendouree-vic/",
  "https://www.stonerealestate.com.au/property/8554111-4-harwood-avenue-mount-kuring-gai-nsw/",
  "https://www.stonerealestate.com.au/property/8554183-1-32-dening-street-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8554194-204-39-george-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8554303-86-kalua-drive-chittaway-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8554388-25-david-street-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8554510-63-christopher-avenue-valentine-nsw/",
  "https://www.stonerealestate.com.au/property/8554626-18-warrawilla-road-wyoming-nsw/",
  "https://www.stonerealestate.com.au/property/8554915-level-10-1045-1-finch-drive-eastgardens-nsw/",
  "https://www.stonerealestate.com.au/property/8555320-13-23-glenvale-close-west-pennant-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8555830-11-10-kalinya-street-newport-nsw/",
  "https://www.stonerealestate.com.au/property/8555914-79-hirst-street-arncliffe-nsw/",
  "https://www.stonerealestate.com.au/property/8555917-19-malcolm-cole-terrace-whitlam-act/",
  "https://www.stonerealestate.com.au/property/8556091-29-alana-drive-west-pennant-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8556623-5-25-waine-street-freshwater-nsw/",
  "https://www.stonerealestate.com.au/property/8556801-14-10-14-bruce-street-brighton-le-sands-nsw/",
  "https://www.stonerealestate.com.au/property/8556887-28-edith-street-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8557548-51-werona-avenue-gordon-nsw/",
  "https://www.stonerealestate.com.au/property/8557740-14-andrew-thompson-drive-mcgraths-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8557805-39-bruce-dittmar-street-forde-act/",
  "https://www.stonerealestate.com.au/property/8557872-25-240-ben-boyd-road-cremorne-nsw/",
  "https://www.stonerealestate.com.au/property/8557873-11-174-spit-road-mosman-nsw/",
  "https://www.stonerealestate.com.au/property/8557948-84-whalans-road-greystanes-nsw/",
  "https://www.stonerealestate.com.au/property/8557951-33-ross-street-windsor-nsw/",
  "https://www.stonerealestate.com.au/property/8558152-1401a-1-17-elsie-street-burwood-nsw/",
  "https://www.stonerealestate.com.au/property/8558193-4-gilgandra-street-lake-heights-nsw/",
  "https://www.stonerealestate.com.au/property/8559215-4-74-80-woniora-road-hurstville-nsw/",
  "https://www.stonerealestate.com.au/property/8559222-10-riversford-close-menangle-nsw/",
  "https://www.stonerealestate.com.au/property/8559307-62-tindara-drive-sawtell-nsw/",
  "https://www.stonerealestate.com.au/property/8559535-13-cousins-road-beacon-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8559770-47-jorgensen-street-moncrieff-act/",
  "https://www.stonerealestate.com.au/property/8559771-60-pademelon-street-throsby-act/",
  "https://www.stonerealestate.com.au/property/8560008-4b-shore-break-crescent-lake-cathie-nsw/",
  "https://www.stonerealestate.com.au/property/8560068-35a-king-street-manly-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8560088-2903-330-church-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8560189-6a-tessa-street-chatswood-nsw/",
  "https://www.stonerealestate.com.au/property/8560551-7-688-rocky-point-road-sans-souci-nsw/",
  "https://www.stonerealestate.com.au/property/8560557-23-tunbridge-place-jannali-nsw/",
  "https://www.stonerealestate.com.au/property/8560810-10-castle-court-kallangur-qld/",
  "https://www.stonerealestate.com.au/property/8561040-9-gleneagles-avenue-killara-nsw/",
  "https://www.stonerealestate.com.au/property/8561042-35-505-507-rocky-point-road-sans-souci-nsw/",
  "https://www.stonerealestate.com.au/property/8561047-42-mangrove-road-narara-nsw/",
  "https://www.stonerealestate.com.au/property/8561195-36-24-flinders-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8561349-28-kurrawyba-avenue-terrigal-nsw/",
  "https://www.stonerealestate.com.au/property/8561577-371-newport-road-cooranbong-nsw/",
  "https://www.stonerealestate.com.au/property/8561679-8-reef-street-nirimba-fields-quakers-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8561749-33-adelaide-street-balgowlah-heights-nsw/",
  "https://www.stonerealestate.com.au/property/8561954-4a-low-street-mount-kuring-gai-nsw/",
  "https://www.stonerealestate.com.au/property/8562124-10-narramore-street-kingsgrove-nsw/",
  "https://www.stonerealestate.com.au/property/8562384-2-aria-entrance-rothbury-nsw/",
  "https://www.stonerealestate.com.au/property/8562388-29-penfold-street-gungahlin-act/",
  "https://www.stonerealestate.com.au/property/8562389-12-miago-court-ngunnawal-act/",
  "https://www.stonerealestate.com.au/property/8562776-60-hawkesbury-drive-donnybrook-vic/",
  "https://www.stonerealestate.com.au/property/8562845-88-burns-road-kellyville-nsw/",
  "https://www.stonerealestate.com.au/property/8562923-12-pheasant-street-toronto-nsw/",
  "https://www.stonerealestate.com.au/property/8563114-4-raymond-street-tacoma-south-nsw/",
  "https://www.stonerealestate.com.au/property/8563690-3-19-oak-circuit-raymond-terrace-nsw/",
  "https://www.stonerealestate.com.au/property/8563880-374-mandalay-circuit-beveridge-vic/",
  "https://www.stonerealestate.com.au/property/8564239-45-burgess-street-beverley-park-nsw/",
  "https://www.stonerealestate.com.au/property/8564681-4-2-factory-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8564754-6-elizabeth-street-riverstone-nsw/",
  "https://www.stonerealestate.com.au/property/8564979-3-brierley-place-eagle-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8565052-6-111-harbord-road-freshwater-nsw/",
  "https://www.stonerealestate.com.au/property/8565166-2-21-heyes-street-gillieston-heights-nsw/",
  "https://www.stonerealestate.com.au/property/8565182-40-brabyn-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8565241-16-clark-road-noraville-nsw/",
  "https://www.stonerealestate.com.au/property/8565250-185a-cumberland-road-greystanes-nsw/",
  "https://www.stonerealestate.com.au/property/8565291-70-dunning-avenue-rosebery-nsw/",
  "https://www.stonerealestate.com.au/property/8565326-654-ourimbah-creek-road-palm-grove-nsw/",
  "https://www.stonerealestate.com.au/property/8565737-24-9-banksia-avenue-banksia-nsw/",
  "https://www.stonerealestate.com.au/property/8565937-37-bundilla-parade-berkeley-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8565938-103-70-efkarpidis-street-gungahlin-act/",
  "https://www.stonerealestate.com.au/property/8566017-25-chippindall-street-speers-point-nsw/",
  "https://www.stonerealestate.com.au/property/8566742-97-rebecca-parade-winston-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8566795-3-17-buckland-street-balgownie-nsw/",
  "https://www.stonerealestate.com.au/property/8566796-9-saltwater-circuit-kanahooka-nsw/",
  "https://www.stonerealestate.com.au/property/8566856-4-8-james-street-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8566857-10-1-3-wentworth-street-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8566864-4-35-37-ross-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8567063-30-972-990-old-princes-highway-engadine-nsw/",
  "https://www.stonerealestate.com.au/property/8567064-33-oakland-avenue-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8567209-46-robsons-road-keiraville-nsw/",
  "https://www.stonerealestate.com.au/property/8567274-3-45-cessnock-road-weston-nsw/",
  "https://www.stonerealestate.com.au/property/8567341-5-23-great-western-highway-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8567830-40-mcgee-avenue-wamberal-nsw/",
  "https://www.stonerealestate.com.au/property/8567890-101-aloha-drive-chittaway-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8567976-22-daley-loop-harrington-park-nsw/",
  "https://www.stonerealestate.com.au/property/8568063-20a-arlington-street-gorokan-nsw/",
  "https://www.stonerealestate.com.au/property/8568126-25-kirkwall-avenue-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8568283-level-4-33-2a-brady-street-mosman-nsw/",
  "https://www.stonerealestate.com.au/property/8568402-704-23-george-street-burwood-nsw/",
  "https://www.stonerealestate.com.au/property/8568727-123-koloona-avenue-mount-keira-nsw/",
  "https://www.stonerealestate.com.au/property/8568766-15-woods-street-redhead-nsw/",
  "https://www.stonerealestate.com.au/property/8568799-47-harrow-circuit-greta-nsw/",
  "https://www.stonerealestate.com.au/property/8569119-802-36-baker-street-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8569593-43-235-flemington-road-franklin-act/",
  "https://www.stonerealestate.com.au/property/8569736-102-50-54-mallett-street-camperdown-nsw/",
  "https://www.stonerealestate.com.au/property/8569760-40-mills-crescent-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8569779-22-pindari-drive-dunbogan-nsw/",
  "https://www.stonerealestate.com.au/property/8569783-100-atchison-street-crows-nest-nsw/",
  "https://www.stonerealestate.com.au/property/8569846-3-13-margaret-street-belmont-north-nsw/",
  "https://www.stonerealestate.com.au/property/8570002-803-20-kendall-street-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8570070-55-byron-court-phillip-act/",
  "https://www.stonerealestate.com.au/property/8570438-g1-39-moffatts-drive-dundas-valley-nsw/",
  "https://www.stonerealestate.com.au/property/8570454-9-katoomba-street-harrison-act/",
  "https://www.stonerealestate.com.au/property/8570526-7-silverdale-parade-jewells-nsw/",
  "https://www.stonerealestate.com.au/property/8570539-1a-wild-flower-place-tahmoor-nsw/",
  "https://www.stonerealestate.com.au/property/8570727-30-open-drive-snug-tas/",
  "https://www.stonerealestate.com.au/property/8571126-49-kings-canyon-street-harrison-act/",
  "https://www.stonerealestate.com.au/property/8571366-167-granite-street-port-macquarie-nsw/",
  "https://www.stonerealestate.com.au/property/8571402-20a-craigie-ave-kanwal-nsw/",
  "https://www.stonerealestate.com.au/property/8571693-24-261-condamine-street-manly-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8571697-8-wyong-road-tumbi-umbi-nsw/",
  "https://www.stonerealestate.com.au/property/8571735-17-campbell-street-north-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8571784-19-long-reef-crescent-woodbine-nsw/",
  "https://www.stonerealestate.com.au/property/8571876-30a-pass-avenue-thirroul-nsw/",
  "https://www.stonerealestate.com.au/property/8571893-34-rosella-street-nicholls-act/",
  "https://www.stonerealestate.com.au/property/8572041-7-mangrove-close-lakewood-nsw/",
  "https://www.stonerealestate.com.au/property/8572188-11-knight-street-arncliffe-nsw/",
  "https://www.stonerealestate.com.au/property/8572399-36-143-149-corrimal-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8572710-80-aspinall-street-watson-act/",
  "https://www.stonerealestate.com.au/property/8572770-25a-cohen-way-thrumster-nsw/",
  "https://www.stonerealestate.com.au/property/8572845-717-15-thompsons-road-pokolbin-nsw/",
  "https://www.stonerealestate.com.au/property/8572866-26-scarborough-way-dunbogan-nsw/",
  "https://www.stonerealestate.com.au/property/8572923-32-prince-street-bellbird-nsw/",
  "https://www.stonerealestate.com.au/property/8572945-ourimbah-nsw/",
  "https://www.stonerealestate.com.au/property/8572975-2-lewis-street-epping-nsw/",
  "https://www.stonerealestate.com.au/property/8573206-20-enright-drive-north-rothbury-nsw/",
  "https://www.stonerealestate.com.au/property/8573263-71-serafina-drive-helensvale-qld/",
  "https://www.stonerealestate.com.au/property/8573648-402-888b-pacific-highway-gordon-nsw/",
  "https://www.stonerealestate.com.au/property/8573713-206-100-bay-road-waverton-nsw/",
  "https://www.stonerealestate.com.au/property/8573877-38-peppercorn-avenue-narellan-nsw/",
  "https://www.stonerealestate.com.au/property/8574067-20-francis-street-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8574116-9-owen-avenue-kyeemagh-nsw/",
  "https://www.stonerealestate.com.au/property/8574530-14-popran-way-blue-haven-nsw/",
  "https://www.stonerealestate.com.au/property/8574601-17-47-victoria-road-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8574604-9-djerrkurra-street-bonner-act/",
  "https://www.stonerealestate.com.au/property/8574720-21-trinity-point-drive-morisset-park-nsw/",
  "https://www.stonerealestate.com.au/property/8574726-5-234-240-the-entrance-long-jetty-nsw/",
  "https://www.stonerealestate.com.au/property/8574728-17-plantation-place-avoca-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8574931-12-pioneer-street-amaroo-act/",
  "https://www.stonerealestate.com.au/property/8575016-25-166-russell-avenue-dolls-point-nsw/",
  "https://www.stonerealestate.com.au/property/8575019-12-11-15-villiers-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8575081-2-7-spectrum-road-north-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8575129-60-66-mclennans-road-plenty-vic/",
  "https://www.stonerealestate.com.au/property/8575162-184-mount-keira-road-mount-keira-nsw/",
  "https://www.stonerealestate.com.au/property/8575167-20-preddys-road-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/8575384-5-6-parsonage-road-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8575385-13-29-31-muriel-street-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8575435-32-park-street-swansea-nsw/",
  "https://www.stonerealestate.com.au/property/8575543-66-23-regent-honeyeater-grv-north-kellyville-nsw/",
  "https://www.stonerealestate.com.au/property/8575618-28-carlisle-crescent-beecroft-nsw/",
  "https://www.stonerealestate.com.au/property/8575754-71-queen-street-narellan-nsw/",
  "https://www.stonerealestate.com.au/property/8575805-107c-80-waterloo-road-macquarie-park-nsw/",
  "https://www.stonerealestate.com.au/property/8575990-14-anglo-road-greenwich-nsw/",
  "https://www.stonerealestate.com.au/property/8576889-25-baikie-crescent-charlestown-nsw/",
  "https://www.stonerealestate.com.au/property/8577584-32-16-20-park-avenue-waitara-nsw/",
  "https://www.stonerealestate.com.au/property/8577655-25-chepana-street-lake-cathie-nsw/",
  "https://www.stonerealestate.com.au/property/8577703-33-24-26-watt-street-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8577752-12-circulo-drive-copacabana-nsw/",
  "https://www.stonerealestate.com.au/property/8577865-71a-queen-street-narellan-nsw/",
  "https://www.stonerealestate.com.au/property/8578021-5-tununda-place-eschol-park-nsw/",
  "https://www.stonerealestate.com.au/property/8578050-54-91-95-john-whiteway-drive-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8578051-32-kilpa-road-wyongah-nsw/",
  "https://www.stonerealestate.com.au/property/8578133-709-3-pretoria-parade-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8578316-5-ray-street-toukley-nsw/",
  "https://www.stonerealestate.com.au/property/8578350-19-lakeland-circuit-harrington-park-nsw/",
  "https://www.stonerealestate.com.au/property/8578352-7-66-ernest-street-crows-nest-nsw/",
  "https://www.stonerealestate.com.au/property/8578359-2-13-willow-place-port-macquarie-nsw/",
  "https://www.stonerealestate.com.au/property/8578376-3-packard-close-killarney-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8578402-28-5-15-lamond-drive-turramurra-nsw/",
  "https://www.stonerealestate.com.au/property/8578407-91-villiers-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8578461-28-koorabel-avenue-copacabana-nsw/",
  "https://www.stonerealestate.com.au/property/8578513-22-triller-street-aberglasslyn-nsw/",
  "https://www.stonerealestate.com.au/property/8578517-6-8-factory-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8578572-18-edden-street-bellbird-nsw/",
  "https://www.stonerealestate.com.au/property/8578697-82-tozer-street-west-kempsey-nsw/",
  "https://www.stonerealestate.com.au/property/8578731-7-yuruga-place-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8578775-1802-1-boys-avenue-blacktown-nsw/",
  "https://www.stonerealestate.com.au/property/8578814-17b-88-98-marsden-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8579119-30-oswald-street-wilton-nsw/",
  "https://www.stonerealestate.com.au/property/8579258-5305-1a-morton-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8579262-21-23-reilly-street-liverpool-nsw/",
  "https://www.stonerealestate.com.au/property/8579416-813-19-partridge-avenue-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8579426-30a-oswald-street-wilton-nsw/",
  "https://www.stonerealestate.com.au/property/8579469-61-cultivation-road-austral-nsw/",
  "https://www.stonerealestate.com.au/property/8579516-9-jefferis-avenue-cobbitty-nsw/",
  "https://www.stonerealestate.com.au/property/8579653-32-19-21-central-coast-highway-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8579661-7-lytton-place-campbelltown-nsw/",
  "https://www.stonerealestate.com.au/property/8579783-57-astoria-road-wollert-vic/",
  "https://www.stonerealestate.com.au/property/8580033-46-oneill-street-brighton-le-sands-nsw/",
  "https://www.stonerealestate.com.au/property/8580034-5-curlew-crescent-eagleby-qld/",
  "https://www.stonerealestate.com.au/property/8580073-406-10-fielder-street-west-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8580107-35-colonial-street-campbelltown-nsw/",
  "https://www.stonerealestate.com.au/property/8580108-3-83-saywell-road-macquarie-fields-nsw/",
  "https://www.stonerealestate.com.au/property/8580241-12-wilfrid-street-macquarie-fields-nsw/",
  "https://www.stonerealestate.com.au/property/8580271-6-8-12-bourke-street-waterford-west-qld/",
  "https://www.stonerealestate.com.au/property/8580272-7-10-monash-road-loganlea-qld/",
  "https://www.stonerealestate.com.au/property/8580278-6a-peter-place-bligh-park-nsw/",
  "https://www.stonerealestate.com.au/property/8580361-36a-kaleula-crescent-kiama-nsw/",
  "https://www.stonerealestate.com.au/property/8580426-8-pitt-street-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8580453-68-pollock-avenue-wyong-nsw/",
  "https://www.stonerealestate.com.au/property/8580537-27-13-19-bryant-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8580562-2-4-virginia-street-north-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8580601-46-norman-street-mangerton-nsw/",
  "https://www.stonerealestate.com.au/property/8580637-1-125-lakin-street-bateau-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8580918-82-bloomfield-street-south-kempsey-nsw/",
  "https://www.stonerealestate.com.au/property/8580953-50b-blue-crane-close-tumbi-umbi-nsw/",
  "https://www.stonerealestate.com.au/property/8581076-3-brodie-circle-baulkham-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8581178-18-altitude-street-north-richmond-nsw/",
  "https://www.stonerealestate.com.au/property/8581444-5-olive-grove-lenah-valley-tas/",
  "https://www.stonerealestate.com.au/property/8581636-15-adrian-close-bateau-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8581983-302-41a-manchester-drive-schofields-nsw/",
  "https://www.stonerealestate.com.au/property/8582162-26-second-avenue-north-warrawong-nsw/",
  "https://www.stonerealestate.com.au/property/8582169-3507-24-the-lanes-boulevard-mermaid-waters-qld/",
  "https://www.stonerealestate.com.au/property/8582300-92-whiting-street-labrador-qld/",
  "https://www.stonerealestate.com.au/property/8582476-9-46-48-keira-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8582493-2-74-robsons-road-keiraville-nsw/",
  "https://www.stonerealestate.com.au/property/8582511-4-14-amess-place-belconnen-act/",
  "https://www.stonerealestate.com.au/property/8582512-28-salisbury-street-casey-act/",
  "https://www.stonerealestate.com.au/property/8582520-42-durack-street-downer-act/",
  "https://www.stonerealestate.com.au/property/8582542-18-2-6-noel-street-north-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8582543-4-20-grey-street-keiraville-nsw/",
  "https://www.stonerealestate.com.au/property/8582548-42-59-67-corrimal-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8582717-2-35-waterford-street-kellyville-ridge-nsw/",
  "https://www.stonerealestate.com.au/property/8582754-606-102-swain-street-gungahlin-act/",
  "https://www.stonerealestate.com.au/property/8582821-4-clementine-place-mardi-nsw/",
  "https://www.stonerealestate.com.au/property/8582887-28a-reycroft-avenue-quakers-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8582903-52-gladys-street-rydalmere-nsw/",
  "https://www.stonerealestate.com.au/property/8583077-132-caves-beach-road-caves-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8583100-2-25-27-villiers-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8583102-67-wonga-road-yowie-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8583103-8-67-warrangarree-drive-woronora-heights-nsw/",
  "https://www.stonerealestate.com.au/property/8583144-28-debono-drive-miners-rest-vic/",
  "https://www.stonerealestate.com.au/property/8583167-32-frances-street-northmead-nsw/",
  "https://www.stonerealestate.com.au/property/8583238-26-21-ijong-street-braddon-act/",
  "https://www.stonerealestate.com.au/property/8583239-27-1-beissel-street-belconnen-act/",
  "https://www.stonerealestate.com.au/property/8583255-26a-bunbury-road-macquarie-fields-nsw/",
  "https://www.stonerealestate.com.au/property/8583311-24a-pillar-street-bellbird-nsw/",
  "https://www.stonerealestate.com.au/property/8583406-959-the-entrance-road-forresters-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8583408-8-ivy-avenue-chain-valley-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8583409-1-15-oaks-avenue-dee-why-nsw/",
  "https://www.stonerealestate.com.au/property/8583410-8-dahlia-avenue-hamlyn-terrace-nsw/",
  "https://www.stonerealestate.com.au/property/8584155-3-87-caldarra-avenue-engadine-nsw/",
  "https://www.stonerealestate.com.au/property/8584159-5-38-morton-street-wollstonecraft-nsw/",
  "https://www.stonerealestate.com.au/property/8584243-24-43-railway-parade-engadine-nsw/",
  "https://www.stonerealestate.com.au/property/8584329-20-ferguson-street-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8584336-1604-330-church-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8584349-1-27a-old-gympie-road-kallangur-qld/",
  "https://www.stonerealestate.com.au/property/8584402-8-33-yarrawonga-street-ngunnawal-act/",
  "https://www.stonerealestate.com.au/property/8584435-6-20-victoria-road-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8584437-17-73-75-flora-street-kirrawee-nsw/",
  "https://www.stonerealestate.com.au/property/8584452-11-127-duffield-road-kallangur-qld/",
  "https://www.stonerealestate.com.au/property/8584571-116a-river-road-greenwich-nsw/",
  "https://www.stonerealestate.com.au/property/8584624-12-dwon-street-wilton-nsw/",
  "https://www.stonerealestate.com.au/property/8584711-52-neptune-street-umina-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8584713-814-120-eastern-valley-way-belconnen-act/",
  "https://www.stonerealestate.com.au/property/8584714-12a-dwon-street-wilton-nsw/",
  "https://www.stonerealestate.com.au/property/8584724-2-12-rangers-road-cremorne-nsw/",
  "https://www.stonerealestate.com.au/property/8584725-level-3-3d-4-distillery-drive-pyrmont-nsw/",
  "https://www.stonerealestate.com.au/property/8584777-13-highcliff-crescent-north-lakes-qld/",
  "https://www.stonerealestate.com.au/property/8584785-113-1-sergeants-lane-st-leonards-nsw/",
  "https://www.stonerealestate.com.au/property/8584786-8c-3-jersey-road-artarmon-nsw/",
  "https://www.stonerealestate.com.au/property/8584787-5-43-quarry-road-dundas-nsw/",
  "https://www.stonerealestate.com.au/property/8584820-48-lord-street-roseville-nsw/",
  "https://www.stonerealestate.com.au/property/8584841-2-338-scarborough-road-scarborough-qld/",
  "https://www.stonerealestate.com.au/property/8584845-2-29-austin-street-fairlight-nsw/",
  "https://www.stonerealestate.com.au/property/8584901-45-florence-street-towradgi-nsw/",
  "https://www.stonerealestate.com.au/property/8584902-23-derribong-drive-cordeaux-heights-nsw/",
  "https://www.stonerealestate.com.au/property/8584909-5-bullecourt-avenue-engadine-nsw/",
  "https://www.stonerealestate.com.au/property/8584915-84-22-peel-street-kirribilli-nsw/",
  "https://www.stonerealestate.com.au/property/8584962-6-brissendon-close-tarrawanna-nsw/",
  "https://www.stonerealestate.com.au/property/8584963-278-maitland-road-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8584970-25-moorlan-ave-killarney-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8585019-52-54-murray-park-road-figtree-nsw/",
  "https://www.stonerealestate.com.au/property/8585020-41-butterfactory-drive-calderwood-nsw/",
  "https://www.stonerealestate.com.au/property/8585022-1-21-arrow-avenue-figtree-nsw/",
  "https://www.stonerealestate.com.au/property/8585031-1-26-king-street-umina-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8585102-24b-breynia-street-figtree-nsw/",
  "https://www.stonerealestate.com.au/property/8585103-202-47-51-crown-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8585104-23-meadow-street-tarrawanna-nsw/",
  "https://www.stonerealestate.com.au/property/8585199-4-devlin-street-gungahlin-act/",
  "https://www.stonerealestate.com.au/property/8585270-23-congewai-street-kearsley-nsw/",
  "https://www.stonerealestate.com.au/property/8585279-32-3809-pacific-highway-tanah-merah-qld/",
  "https://www.stonerealestate.com.au/property/8585335-34-james-street-south-windsor-nsw/",
  "https://www.stonerealestate.com.au/property/8585434-4-abercorn-street-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/8585487-36-main-street-smithtown-nsw/",
  "https://www.stonerealestate.com.au/property/8585546-5-38-40-harold-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8585589-14-55-57-hassall-street-westmead-nsw/",
  "https://www.stonerealestate.com.au/property/8585646-22-sturt-street-swansea-nsw/",
  "https://www.stonerealestate.com.au/property/8585649-50-38-orara-street-waitara-nsw/",
  "https://www.stonerealestate.com.au/property/8585779-26b-connel-drive-heddon-greta-nsw/",
  "https://www.stonerealestate.com.au/property/8585781-9-12-linda-street-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8585802-208-19-parramatta-road-homebush-nsw/",
  "https://www.stonerealestate.com.au/property/8585858-5-quintero-close-heddon-greta-nsw/",
  "https://www.stonerealestate.com.au/property/8585927-27-berkeley-street-speers-point-nsw/",
  "https://www.stonerealestate.com.au/property/8585930-28-fuller-avenue-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8585937-20-guernsey-avenue-minto-nsw/",
  "https://www.stonerealestate.com.au/property/8585987-6-605-king-street-newtown-nsw/",
  "https://www.stonerealestate.com.au/property/8586072-151-hampshire-boulevard-spring-farm-nsw/",
  "https://www.stonerealestate.com.au/property/8586143-4-14-victoria-parade-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8586144-5-bowral-crescent-tullimbar-nsw/",
  "https://www.stonerealestate.com.au/property/8586146-6-heidi-place-west-pennant-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8586153-30-2-pound-road-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8586154-8-53-prospect-street-rosehill-nsw/",
  "https://www.stonerealestate.com.au/property/8586155-3-9-11-priddle-street-westmead-nsw/",
  "https://www.stonerealestate.com.au/property/8586159-14-john-hunter-grove-mount-annan-nsw/",
  "https://www.stonerealestate.com.au/property/8586223-46-pioneer-road-bellambi-nsw/",
  "https://www.stonerealestate.com.au/property/8586226-16-albert-street-north-lambton-nsw/",
  "https://www.stonerealestate.com.au/property/8586298-7-316-318-pennant-hills-road-carlingford-nsw/",
  "https://www.stonerealestate.com.au/property/8586305-4-garrard-street-spence-act/",
  "https://www.stonerealestate.com.au/property/8586355-1-luck-street-slacks-creek-qld/",
  "https://www.stonerealestate.com.au/property/8586465-24-keswick-avenue-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8586475-1401-41-45-waitara-avenue-waitara-nsw/",
  "https://www.stonerealestate.com.au/property/8586573-2-77-ruttleys-road-wyee-nsw/",
  "https://www.stonerealestate.com.au/property/8586574-13-caudalie-street-bellbird-nsw/",
  "https://www.stonerealestate.com.au/property/8586659-7-11-stuart-street-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8586660-11-46-48-solander-street-monterey-nsw/",
  "https://www.stonerealestate.com.au/property/8586662-411-pacific-highway-asquith-nsw/",
  "https://www.stonerealestate.com.au/property/8586668-5-174-gertrude-street-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8586677-18-11-17-watson-street-neutral-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8586709-3-28-36-nursery-street-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8586888-204-32-34-ferntree-place-epping-nsw/",
  "https://www.stonerealestate.com.au/property/8586912-19-15-17-parc-guell-drive-campbelltown-nsw/",
  "https://www.stonerealestate.com.au/property/8586917-12-morrison-close-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8586923-2-5-bruce-street-ashfield-nsw/",
  "https://www.stonerealestate.com.au/property/8587005-737-princes-highway-blakehurst-nsw/",
  "https://www.stonerealestate.com.au/property/8587027-15-46-48-bridge-road-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8587029-5a-waterside-close-point-clare-nsw/",
  "https://www.stonerealestate.com.au/property/8587031-1914-3-grazier-lane-belconnen-act/",
  "https://www.stonerealestate.com.au/property/8587033-3-18-boronia-street-wollstonecraft-nsw/",
  "https://www.stonerealestate.com.au/property/8587037-6-12-star-street-sandy-bay-tas/",
  "https://www.stonerealestate.com.au/property/8587039-4-18-torrens-avenue-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8587104-717-22-central-avenue-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8587114-3-william-cox-drive-richmond-nsw/",
  "https://www.stonerealestate.com.au/property/8587180-383-randalls-bay-road-eggs-and-bacon-bay-tas/",
  "https://www.stonerealestate.com.au/property/8587238-303-2d-appleroth-street-melrose-park-nsw/",
  "https://www.stonerealestate.com.au/property/8587280-9a-australia-road-barden-ridge-nsw/",
  "https://www.stonerealestate.com.au/property/8587359-10a-ferndale-street-chatswood-nsw/",
  "https://www.stonerealestate.com.au/property/8587360-4-22-hickey-street-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8587452-level-g-5-16-hudson-street-lewisham-nsw/",
  "https://www.stonerealestate.com.au/property/8587588-3-11-powell-road-blackmans-bay-tas/",
  "https://www.stonerealestate.com.au/property/8587740-8-102-oconnell-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8587749-45-jacks-creek-road-whittlesea-vic/",
  "https://www.stonerealestate.com.au/property/8587784-8j-15-campbell-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8587825-48-stanford-street-kurri-kurri-nsw/",
  "https://www.stonerealestate.com.au/property/8587867-6-westerman-street-casey-act/",
  "https://www.stonerealestate.com.au/property/8587879-3-airdsley-lane-bradbury-nsw/",
  "https://www.stonerealestate.com.au/property/8587930-22-sunrise-drive-boambee-east-nsw/",
  "https://www.stonerealestate.com.au/property/8587971-500-bresnehans-road-little-swanport-tas/",
  "https://www.stonerealestate.com.au/property/8588022-77-mackays-road-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8588026-311-redwood-road-kingston-tas/",
  "https://www.stonerealestate.com.au/property/8588072-71-buller-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8588074-56-178-184-ewing-road-logan-central-qld/",
  "https://www.stonerealestate.com.au/property/8588075-21-60-macarthy-road-marsden-qld/",
  "https://www.stonerealestate.com.au/property/8588088-11-roslyn-avenue-kingston-beach-tas/",
  "https://www.stonerealestate.com.au/property/8588238-35-mayfield-street-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8588316-35-8-ken-tribe-street-coombs-act/",
  "https://www.stonerealestate.com.au/rent/",
  "https://www.stonerealestate.com.au/rent/commercial-lease/",
  "https://www.stonerealestate.com.au/rent/leasing-with-stone/",
  "https://www.stonerealestate.com.au/rent/property-alerts/",
  "https://www.stonerealestate.com.au/rent/request-appraisal/",
  "https://www.stonerealestate.com.au/rent/residential-lease/",
  "https://www.stonerealestate.com.au/sell/request-appraisal/",
  "https://www.stonerealestate.com.au/sell/selling-with-stone/",
  "https://www.stonerealestate.com.au/sold/",
  "https://www.stonerealestate.com.au/stone-tv-2/",
  "https://www.stonerealestate.com.au/strata/",
  "https://www.stonerealestate.com.au/terms-and-conditions/"
]

In [ ]:
#@title pages extractor
import os
import re
import requests
import trafilatura
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import shutil

# -----------------------------
# Helpers
# -----------------------------
def clean_filename(name):
    """Clean a string so it is safe as a filename."""
    name = re.sub(r'[^a-zA-Z0-9_-]', '_', name)
    name = re.sub(r'_+', '_', name)
    return name.strip("_")


def title_from_html(html):
    """Extract <title> and clean it."""
    soup = BeautifulSoup(html, "html.parser")
    title = soup.title.string if soup.title else "untitled_page"
    return clean_filename(title)


# -----------------------------
# Main Extraction Logic
# -----------------------------
output_dir = "stonerealestate"
zip_name = "stonerealestate"

os.makedirs(output_dir, exist_ok=True)

for url in pages:
    try:
        print(f"[INFO] Fetching: {url}")
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        html_source = response.text

        # Extract title for both filenames
        title_clean = title_from_html(html_source)

        # -----------------------------
        # Save Raw HTML (title.html)
        # -----------------------------
        html_filename = f"{title_clean}.html"
        html_filepath = os.path.join(output_dir, html_filename)

        with open(html_filepath, "w", encoding="utf-8") as f:
            f.write(html_source)

        print(f"[HTML] Saved: {html_filepath}")

        # -----------------------------
        # Save Extracted Text (title.txt)
        # -----------------------------
        extracted_text = trafilatura.extract(html_source)
        if not extracted_text:
            print(f"[WARN] Could not extract main content from {url}")
            continue

        txt_filename = f"{title_clean}.txt"
        txt_filepath = os.path.join(output_dir, txt_filename)

        with open(txt_filepath, "w", encoding="utf-8") as f:
            f.write(f"{title_clean}\n\n")  # also add title inside the text
            f.write(extracted_text)

        print(f"[TEXT] Saved: {txt_filepath}")

    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Failed to fetch {url} -> {e}")

# -----------------------------
# Zip the output directory
# -----------------------------
shutil.make_archive(zip_name, 'zip', output_dir)
print(f"\n[FINAL] All extracted files zipped into: {zip_name}.zip")

[INFO] Fetching: https://www.stonerealestate.com.au
[HTML] Saved: stonerealestate/Stone_Real_Estate_Real_Estate_Agent_Property_Managers.html
[TEXT] Saved: stonerealestate/Stone_Real_Estate_Real_Estate_Agent_Property_Managers.txt
[INFO] Fetching: https://www.stonerealestate.com.au/about/
[HTML] Saved: stonerealestate/Stone_Real_Estate_About_Us_Stone_Real_Estate.html
[TEXT] Saved: stonerealestate/Stone_Real_Estate_About_Us_Stone_Real_Estate.txt
[INFO] Fetching: https://www.stonerealestate.com.au/acreage/
[HTML] Saved: stonerealestate/Stone_Real_Estate_Acreage_Stone_Real_Estate.html
[TEXT] Saved: stonerealestate/Stone_Real_Estate_Acreage_Stone_Real_Estate.txt
[INFO] Fetching: https://www.stonerealestate.com.au/blog/
[HTML] Saved: stonerealestate/Stone_Real_Estate_Blog_Stone_Real_Estate.html
[TEXT] Saved: stonerealestate/Stone_Real_Estate_Blog_Stone_Real_Estate.txt
[INFO] Fetching: https://www.stonerealestate.com.au/buy/
[HTML] Saved: stonerealestate/Real_Estate_For_Sale_Stone_Real_Estate.